# Attach a vertex

## Procedures

In [1]:
# Load the LinearAlgebra package for vector and matrix operations
with(LinearAlgebra):

# Load Simplicial Surface package
libname := (libname, "/users/stuettgen/custom_maple_libraries"):
read("/users/stuettgen/Documents/Masterarbeit/Maple/Initialization_local.mpl"):

# change Jupyter Renderer Outpu
with(Jupyter):
SetOutputRendererByType(equation, "text/plain"):
SetOutputRendererByType(exprseq, "text/plain"):
SetOutputRendererByType(Vector, "text/plain"):

with(ArrayTools):

Maple Error: Error, invalid input: Jupyter:-SetOutputRendererByType expects its 1st argument, mplType, to be of type type, but received exprseq

*Purpose:* This is the "Solve" step. It performs the actual intersection calculation, but does so in the easy, aligned system where the sphere equations are simple.

*How it Works:*
In the aligned system, the sphere equations are: \
Sphere 1: x² + y² + z² = r₁² \
Sphere 2: (x-d)² + y² + z² = r₂² \
Sphere 3: (x-i)² + (y-j)² + z² = r₃²
- By subtracting the second equation from the first, the y² and z² terms cancel out, leaving a very simple linear equation that can be immediately solved for x.
- Similarly, by using the first and third equations and the now-known value of x, it can easily solve for y.
- With both x and y known, it plugs them back into the simplest equation (from Sphere 1) to solve for z². This gives z = ±sqrt(...).

*Output:* It returns the two local intersection points [x, y, z] and [x, y, -z], which lie on opposite sides of the xy-plane. If the spheres don't intersect, it returns am empty list.

In [2]:
# =============================================================================
# HELPER PROCEDURE 1: Trilaterate
# Calculates intersection in the canonical XY-plane system.
# =============================================================================
Trilaterate := proc(r1, r2, r3, d, i, j)
    local x, y, z_sq, z, p1, p2;
    description "Calculates intersection for spheres in canonical alignment.";

    if d = 0 or j = 0 then
        # This case is handled by AlignTriangle's collinearity check,
        # but is included for robustness.
        error "Sphere centers cannot be collinear.";
    end if;

    x := evala((r1^2 - r2^2 + d^2) / (2 * d));
    y := evala((r1^2 - r3^2 + i^2 + j^2 - 2 * i * x) / (2 * j));
    z_sq := r1^2 - x^2 - y^2;

    # if z_sq < 0 then
    #     return [];
    # elif z_sq = 0 then
    #     return [[x, y, 0.0]]; # Return as a list of one point
    # else
    z := evala(sqrt(z_sq)) assuming r1>=0, r2>=0, r3>=0;
    p1 := Vector([x, y, z]);   # Point with positive z
    p2 := Vector([x, y, -z]);  # Point with negative z
    return [p1, p2];
    # end if;
end proc:

*Purpose:* This is the "Simplify" step. Its job is to calculate the exact rotation and translation needed to move the sphere centers into the simple, canonical orientation.

*How it Works:*
- Translation: It first translates the entire system so that sphereCenter1 is at the origin. The translationVector required is simply the original coordinate vector of sphereCenter1.
- Rotation: This is the core of the procedure. It builds a new set of 3D axes (u,v,w) based on the geometry of the sphere centers' triangle:
    - The new x-axis is defined as the direction from the (translated) sphereCenter1 to sphereCenter2.
    - The new z-axis is defined as the vector perpendicular to the plane of the triangle formed by the three centers. This is found using the CrossProduct.
    - The new y-axis is calculated as the cross product of the new z and x axes to ensure it's perpendicular to both and forms a standard "right-handed" coordinate system.

*Output:* It returns the rotationMatrix and translationVector that represent this transformation. The rotationMatrix is the tool that can convert any point from the original coordinate system into this new, simplified one.

In [3]:
# =============================================================================
# HELPER PROCEDURE 2: AlignTriangle
# Computes the transformation to move sphere centers to the XY-plane.
# =============================================================================
AlignTriangle := proc(p1::Vector, p2::Vector, p3::Vector)
    local v1, v2, u, v, w, R, T, p1_new, p2_new, p3_new, cross_prod_norm;
    description "Computes rigid transformation to align 3 points with XY plane.";

    T := p1;
    v1 := p2 - T;
    v2 := p3 - T;

    if Norm(v1, 2) = 0 then
        error "Sphere centers 1 and 2 are coincident.";
    end if;
    if Norm(CrossProduct(v1, v2), 2) = 0 then
        error "Sphere centers are collinear; cannot define a unique plane.";
    end if;

    u := Normalize(v1, 2);
    w := Normalize(CrossProduct(v1, v2), 2);
    v := CrossProduct(w, u);

    R := Matrix([Vector(u), Vector(v), Vector(w)])^+;
    
    p1_new := R . (p1 - T);
    p2_new := R . (p2 - T);
    p3_new := R . (p3 - T);

    return [p1_new, p2_new, p3_new], R, T;
end proc:

In [4]:
# =============================================================================
#         Generalized Intersection of Three Arbitrary Spheres in 3D
# =============================================================================
#
# This script provides a primary function, IntersectThreeSpheres, which
# computes one of the two possible intersection points of three spheres defined
# by their centers (C1, C2, C3) and radii (r1, r2, r3).
#
# The method works as follows:
# 1. A rigid transformation (rotation + translation) is computed using the
#    helper function `AlignTriangle`. This transformation moves the sphere
#    centers to a simplified, canonical orientation on the XY-plane.
# 2. The intersection is calculated in this simplified system using the helper
#    function `Trilaterate`.
# 3. A `sign_choice` argument (+1 or -1) determines which of the two
#    intersection points (on either side of the triangle's plane) is chosen.
# 4. The chosen intersection point is then mapped back to the original
#    coordinate system using the inverse of the initial transformation.
#
# =============================================================================
IntersectThreeSpheres := proc(C1::Vector, C2::Vector, C3::Vector, r1, r2, r3, sign_choice::{+1,-1})
    local alignment_data, transformed_centers, R, T, d, i, j,
          local_points, selected_point_local, P_local, P_final;

    description "Computes the intersection of three arbitrary spheres.";
          
    # --- Step 1: Align the sphere centers to the XY plane ---
    # The try-catch block will handle collinearity errors from AlignTriangle.
    try
        transformed_centers, R, T := AlignTriangle(C1, C2, C3);
    catch:
        error "Error during alignment: " || lasterror[2];
    end try;

    # --- Step 2: Extract parameters for the canonical solver ---
    # d is the distance between transformed C1 and C2 (it's the x-coord of C2_new)
    d := transformed_centers[2][1];
    # i and j are the coordinates of the transformed C3
    i := transformed_centers[3][1];
    j := transformed_centers[3][2];

    # --- Step 3: Solve for the intersection in the simplified system ---
    local_points := Trilaterate(r1, r2, r3, d, i, j);

    # If Trilaterate returned a string, it means no solution, so propagate it.
    if nops(local_points) = 0 then
        return;
    end if;

    # --- Step 4: Select the desired intersection point ---
    # `local_points` is a list of lists, e.g., [[x,y,z], [x,y,-z]]
    if nops(local_points) = 1 then
        # Tangent case: only one point, sign_choice doesn't matter.
        P_local := local_points[1];
    else
        # Two points exist. The first has z >= 0, the second has z <= 0.
        if sign_choice = +1 then
            P_local := local_points[1]; # Positive z in local system
        else
            P_local := local_points[2]; # Negative z in local system
        end if;
    end if;

    # --- Step 5: Apply the inverse transformation ---
    # To restore the point to its original coordinate system, we first apply
    # the inverse rotation (Transpose(R)) and then the inverse translation (+ T).
    P_final := Transpose(R) . P_local + T;

    return P_final;

end proc:



In [17]:
sqdist := proc(v::Vector, w::Vector) # square distance between two 3d-vectors v,w
    local x;
    x := v-w;
    return evala(add(map(entry -> entry^2, x)));
end proc:

sqnorm := proc(v::Vector) # return the square of the 2-norm of a ed-vector v
    return sqdist(v,Vector([seq(0, 1..nops(v))]))
end proc:

EmbedNextVertex := proc(embedded_coords::List[Vector], neighbor_indices::List[Integer], distances::List, next_param_index::Integer, {sign_choice::{+1,-1} := +1})
    local new_vertex_index, new_vertex_coords, next_param_index, n, neighbor_points, new_embedding;
    
    description "Computes coordinates for a new vertex, including canonical placement for the first three.";
    new_vertex_index := nops(embedded_coords) + 1;

    n := nops(neighbor_indices);

    # --- PART 1: Canonical placement for the first three vertices ---
    if new_vertex_index = 1 then
        new_vertex_coords := Vector([0, 0, 0]);

        return new_vertex_coords, next_param_index;

    elif new_vertex_index = 2 then
        if n <> 1 or neighbor_indices[1] <> 0 then error "Vertex 1 must be a neighbor of vertex 0 only."; end if;
        new_vertex_coords := Vector([distances[1], 0, 0]);
        new_embedding := [[op(embedded_coords), new_vertex_coords]];

        return new_embedding, next_param_index;

    elif new_vertex_index = 3 then
        local d01, d02, d12, v0, v1, x, y, idx0, idx1;
        idx0 := Search(0, neighbor_indices); d02 := distances[idx0];
        idx1 := Search(1, neighbor_indices); d12 := distances[idx1];
        v0 := embedded_coords[1]; v1 := embedded_coords[2];
        d01 := Norm(v1 - v0, 2);
        x := (d01^2 + d02^2 - d12^2) / (2*d01);
        y := sqrt(d02^2 - x^2);
        new_vertex_coords := Vector([x, y, 0]);

        new_embedding := [[op(embedded_coords), new_vertex_coords]];

        return new_embedding, next_param_index;
    end if;

    # --- PART 2: General embedding logic for all subsequent vertices ---
    neighbor_points := embeddec_coords[neighbor_indices];

    if n <= 2 then
        local anchors, radii, p_idx;
        anchors := copy(neighbor_points); radii := copy(distances);
        
        for p_idx from 1 to nops(embedded_coords) do
            if not p_idx in neighbor_points then
                neighbor_indices := [op(neighbor_points), p_idx];
                neighbor_points := embeddec_coords[neighbor_indices];
                distances := [op(distances), _t[next_param_index]]
                next_param_index := next_param_index + 1;
            end if;

            if nops(neighbor_points) = 3 then
                new_vertex_coords := IntersectThreeSpheres(neighbor_points[1], neighbor_points[2], neighbor_points[3], distances[1], distances[2], distances[3], sign_choice);

                new_embedding := [[op(embedded_coords), new_vertex_coords]];

                return new_embedding, next_param_index;
        end do;
        
        if nops(neighbor_points) < 3 then error "Could not find enough unique anchor points."; end if;

    elif n = 3 then
            new_vertex_coords := IntersectThreeSpheres(neighbor_points[1], neighbor_points[2], neighbor_points[3], distances[1], distances[2], distances[3], sign_choice);

            new_embedding := [[op(embedded_coords), new_vertex_coords]];

            return new_embedding, next_param_index;

    else # n >= 4
        new_vertex_coords := IntersectThreeSpheres(neighbor_points[1], neighbor_points[2], neighbor_points[3], distances[1], distances[2], distances[3], sign_choice);

        remaining_points := neighbor_points[4..nops(neighbor_points)];
        remaining_dists := dists[4..nops(dists)];

        coords := [op(embedded_coords), new_vertex_coords];

        # need to solve system of equations to ensure that distance to the remaining neighbors is as desired.
        vars := map(i -> _t[i], [$1..(next_param_index - 1)]);
        eqns := map(i -> sqdist(new_vertex_coords, remaining_points[i]) - remaining_dists[i]^2, [$1..nops(remaining_points)]);
        numer_eqns := evala(numer(eqns));
        denom_eqns := evala(denom(eqns));
        ext := [op(indets(numer_eqn, algext))];
        ext := sort(ext, key = (expr -> nops(indets(expr, algext))));
        ext := [op(map(expr -> [expr], ext))];

        if eqn = [0$nops(eqn)] then
            s := [[]];
        else
            s, u := `SimplicialSurfaceEmbeddings/solve_polynomial_system`(numer_eqns, vars, denom_eqns, ext);
            print(cat("s = ", s));
        end if;

        new_embedding := [];

        for sub in s do
            try
                # print(evala(Simplify~(subs(sub, coordsNew))));
                new_coords := evala(subs(sub, coords));
                # print(evala(dist(coordsNew1[tip], coordsNew1[v])^2 - elng[Search(e, edgesSets)]^2) assuming real);
                # print(coordsNew1);
                # sanity check that all edge lengths are as desired.
                for neigh in neighbor_indices do
                    if not evala(sqdist(new_coords[neigh], new_coords[nops(new_coords)]) - dists[neigh]) = 0 then
                        next sub;
                    end if;
                end do;
                new_embedding := [op(new_embedding), new_coords];
            # catch: next sub;
            end try;
        end do;
        
        return new_embedding, next_param_index;
    end if;
end proc:

Maple Error: missing operator or `;`

## Usage

### Embedding of Tetrahedron with edge length 1

In [12]:
embedding := [];
next_param_index := 1;

embedding, next_param_index := EmbedNextVertex(embedding, [], [], 1, 1);

                             next_param_index := 1

Maple Error: Error, mismatched multiple assignment of 2 variables on the left side and 1 value on the right side

In [13]:
EmbedNextVertex(embedding, [], [], 1, 1);

EmbedNextVertex([],[],[],1,1)